# 02 - 使用pytorch实现线性回归

**Table of contents**<a id='toc0_'></a>    
- [00 导入所需要的包](#toc1_1_)    
  - [01 使用dataset和dataloader导入数据](#toc1_2_)    
  - [02 定义模型](#toc1_3_)    
  - [03 初始化模型参数](#toc1_4_)    
  - [04 损失函数](#toc1_5_)    
  - [05 随机梯度下降](#toc1_6_)    
  - [06 训练](#toc1_7_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[00 导入所需要的包](#toc0_)

In [1]:
from d2l import torch as d2l
import torch
from torch.utils import data
import numpy as np

In [2]:
def synthetic_data(w, b, num_examples):  #@save
    """生成y=Xw+b+噪声"""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

## <a id='toc1_2_'></a>[01 使用dataset和dataloader导入数据](#toc0_)

In [5]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

在Python中，*符号被称为解包运算符，它允许你将一个可迭代对象的多个元素作为单独的参数传递给函数。当在函数调用期间将*放置在可迭代对象之前时，它会解包可迭代对象的元素，将每个元素视为单独的参数。

在这种情况下，data_arrays是一个包含两个元素（features和labels）的元组。通过使用*data_arrays，元组的元素被解包，features和labels被作为单独的参数传递给data.TensorDataset构造函数。这相当于调用data.TensorDataset(features, labels)。

In [6]:
next(iter(data_iter))

[tensor([[-0.8517, -1.4215],
         [ 0.1381,  0.3108],
         [ 0.1346,  0.0951],
         [-0.5294,  0.9226],
         [ 1.0235,  0.1191],
         [-0.5475, -2.2133],
         [-0.1997,  1.2014],
         [-0.1301, -0.0048],
         [ 0.1017,  0.8321],
         [-1.3642,  1.5048]]),
 tensor([[ 7.3146],
         [ 3.4208],
         [ 4.1539],
         [ 0.0185],
         [ 5.8514],
         [10.6395],
         [-0.2851],
         [ 3.9380],
         [ 1.5738],
         [-3.6390]])]

## <a id='toc1_3_'></a>[02 定义模型](#toc0_)

In [8]:
# nn是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1)) # Linear(2, 1) 表示输入有两个feature，输出是一个feature

## <a id='toc1_4_'></a>[03 初始化模型参数](#toc0_)

In [13]:
net[0].weight.data.normal_(0, 0.01) # 随机初始化weight的均值为0，方差为0.01
net[0].bias.data.fill_(0) # 初始化bias为0

tensor([0.])

In [14]:
net[0], net[0].weight, net[0].bias

(Linear(in_features=2, out_features=1, bias=True),
 Parameter containing:
 tensor([[-0.0064,  0.0017]], requires_grad=True),
 Parameter containing:
 tensor([0.], requires_grad=True))

## <a id='toc1_5_'></a>[04 损失函数](#toc0_)

计算均方误差使用的是MSELoss类，也称为$L_2$平方范数。

In [18]:
loss = nn.MSELoss()

## <a id='toc1_6_'></a>[05 随机梯度下降](#toc0_)

In [19]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [24]:
type(net.parameters())

generator

## <a id='toc1_7_'></a>[06 训练](#toc0_)

- `net(X)` 表示对X向量的一次正向传播

- `l.backward()` 表示对loss的一次反向传播，反向传播的过程中进行了梯度的求取

- `trainer.step()` 表示优化器向前一步，之后更新w，b的参数

In [23]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000342
epoch 2, loss 0.000100
epoch 3, loss 0.000099


调用 `trainer.zero_grad()` 的目的是将之前计算得到的梯度归零，以确保每个批次的梯度都是新的、独立的计算结果。如果不清零梯度，那么梯度会在每个批次中累积，导致参数更新不正确。

在`backward`之前必须对梯度进行清零

In [28]:
print(f'w的估计误差: {true_w - net[0].weight.reshape(true_w.shape)}')
print(f'b的估计误差: {true_b - net[0].bias}')

w的估计误差: tensor([-0.0011, -0.0003], grad_fn=<SubBackward0>)
b的估计误差: tensor([-0.0005], grad_fn=<RsubBackward1>)
